### Contents
A Model Without an Activation Function<br>
&emsp;Linear Example<br>
&emsp;Quadratic Example<br>


In [ ]:
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor


### A Model without an Activation Function

In [ ]:
def create_linear_data(weight:float, bias: float) -> tuple[List[float], List[float]]:
    X = [float(x) for x in range(-10, 10)]
    y = [weight*x+bias for x in X]
    X, y = np.array(X, dtype=np.float32), np.array(y, np.float32)
    X = np.reshape(X, (len(X), 1))
    y = np.reshape(y, (len(y), 1))
    return X, y

# We need to reshape X and y so that each row is a single feature/label.
# This is needed for training.
X_train, y_train = create_linear_data(5, 9)

print('X_train Dimenstions:',X_train.ndim)
print('X_train Shape:', X_train.shape)
print(X_train[:2])

print('y_train Dimenstions:',y_train.ndim)
print('y_train Shape:', y_train.shape)
print(y_train[:2])

In [ ]:
def plot_data(x: np.ndarray, y1: np.ndarray, y2: np.ndarray=None) -> None:
    ax = plt.subplots()[1]
    ax.set_xlim(x.min()-5, x.max()+5)
    ax.set_ylim(y1.min()-5, y1.max()+5)
    plt.scatter(x, y1, color='blue')
    if not y2 is None:
        ax.scatter(x, y2, color='red')
    plt.grid(True)
    plt.axhline(color='black')
    plt.axvline(color='black')

plot_data(X_train, y_train)

Design the Model

In [ ]:
class LinearRegressionModel(nn.Module):

    def __init__(self):
        super(LinearRegressionModel, self).__init__()

        self.linear1 = nn.Linear(1, 1, bias=True)

    def forward(self, input, log=False):
        l1 = self.linear1(input)
        if log:
            print('\nLinear 1:', l1, l1.shape)
        return l1


Test the Model with a Single Prediction

In [ ]:
model = LinearRegressionModel()
X = torch.tensor([1], dtype=torch.float32)
prediction = model(X, log=True)
print(X)
print(prediction)

Create a Training Function

In [ ]:

# When you have a large number of iterations over a small training set you are basically
# memorizing your training set.

EPOCHS = 100
LR = 0.001

def train_model(model: nn.Module, X_train, y_train) -> tuple[nn.Module, List]:
    torch.manual_seed(42)
    losses = []
    loss_function = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=LR)

    for epoch in range(EPOCHS):
        total_loss = 0
        for X, y in zip(X_train, y_train):

            # Wrap in tensors
            X_tensor = torch.from_numpy(X)
            y_tensor = torch.from_numpy(y)
            
            # Pytorch accumulates gradients so before passing in a new
            # context (features) you need to zero out the gradients from the 
            # previous context.
            model.zero_grad()
            optimizer.zero_grad()

            # Forward pass - this will get log probabilities for every word 
            # in our vocabulary which is now represented as embeddings.
            prediction = model(X_tensor)

            # Compute the loss.
            # target has to be a list for some reason.
            loss = loss_function(prediction, y_tensor)

            # Backward pass to update the gradients.
            loss.backward()

            # Optimize the parameters
            optimizer.step()

            # Get the loss for this context.
            total_loss += loss.item()
            
        # Save the total loss for this epoch.
        losses.append(total_loss)

    return model, losses

In [ ]:
def print_results(model: nn.Module, losses: list) -> None:
    # Print the losses of every 10th epoch.
    for epoch in range(0, EPOCHS, 10):
        print(epoch, ':', losses[epoch])
    # This will print the very last epoch so we can see the
    # final loss value.
    print(EPOCHS-1, ':', losses[EPOCHS-1])

    for name, parameter in model.named_parameters():
        print(name, parameter.data)

In [ ]:

model = LinearRegressionModel()
model, losses = train_model(model, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
print_results(model, losses)

In [ ]:
predictions = [model(torch.from_numpy(X)).detach().item() for X in X_train]

plot_data(np.array(X_train), np.array(y_train), np.array(predictions))

In [ ]:
print(y_train)
print(predictions)

### A Quadratic Example

In [ ]:
def create_quadratic_data(a:float, b: float, c:float) -> tuple[List[float], List[float]]:
    X = [float(x) for x in range(-10, 11)]
    y = [a*(x**2)+(b*x)+c for x in X]
    X, y = np.array(X, dtype=np.float32), np.array(y, np.float32)
    X = np.reshape(X, (len(X), 1))
    y = np.reshape(y, (len(y), 1))
    return X, y

X_train, y_train = create_quadratic_data(5, 2, 3)

print('X_train Dimenstions:',X_train.ndim)
print('X_train Shape:', X_train.shape)
print(X_train[:2])

print('y_train Dimenstions:',y_train.ndim)
print('y_train Shape:', y_train.shape)
print(y_train[:2])

In [ ]:
plot_data(X_train, y_train)

In [28]:
class QuadraticRegressionModel(nn.Module):

    def __init__(self):
        super(QuadraticRegressionModel, self).__init__()

        self.linear1 = nn.Linear(1, 6, bias=True)
        self.linear2 = nn.Linear(6, 6, bias=True)
        self.linear3 = nn.Linear(6, 1, bias=True)

    def forward(self, input, log=False):
        out = self.linear1(input)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        return out

In [29]:
model = QuadraticRegressionModel()
X = torch.tensor([1], dtype=torch.float32)
prediction = model(X, log=True)
print(X)
print(prediction)

tensor([1.])
tensor([-0.1201], grad_fn=<AddBackward0>)


In [30]:
model = QuadraticRegressionModel()
model, losses = train_model(model, X_train, y_train)

# The loss should decrease with every iteration (epoch) over the training data.
print_results(model, losses)

0 : 17505.414018914104
10 : 15787.651941537857
20 : 18625.464256048203
30 : 16565.235090255737
40 : 15929.981842041016
50 : 16262.23739349842
60 : 17962.81825309992
70 : 15117.124224185944
80 : 16236.015011787415
90 : 17911.38174057007
99 : 16902.339913129807
linear1.weight tensor([[ 1.6168e+04],
        [ 3.6443e-02],
        [ 1.8569e+01],
        [ 2.7119e-02],
        [-4.3282e+00],
        [ 2.0051e+00]])
linear1.bias tensor([-3.2348e+03, -6.3758e-01, -3.7179e+00,  1.6593e+00,  1.4655e+00,
        -8.7136e-01])
linear2.weight tensor([[-5.9438e-01, -6.4062e-02, -3.2608e-01, -6.3070e-01, -7.1281e-01,
         -3.8512e-01],
        [-2.6820e-01, -2.7831e-01, -1.7763e-01, -2.6404e-01, -2.5910e-01,
         -1.4227e-01],
        [-1.0006e+00, -2.8975e-01, -8.4744e-01, -1.4949e+00, -1.5695e+00,
         -9.4669e-01],
        [-8.8702e+00,  2.7753e-01, -3.3916e+00, -1.6955e-01, -3.2623e+00,
         -1.0005e+01],
        [-1.4473e-01, -3.9502e-01, -2.3379e-01, -6.8162e-02, -6.3490e-02,
 

In [27]:
activation_functions = {
    'ELU': nn.ELU(),
    'Hardshrink': nn.Hardshrink(),
    'Hardsigmoin': nn.Hardsigmoid(),
    'Hardswish': nn.Hardswish(),
    'Hardtanh': nn.Hardtanh(),
    'LeadkyReLU': nn.LeakyReLU(),
    'LogSigmoid': nn.LogSigmoid(),
    #'MultiheadAttention': nn.MultiheadAttention(embed_dim, num_heads),
    'PReLU': nn.PReLU(),
    'ReLU': nn.ReLU(),
    'ReLU6': nn.ReLU6(),
    'RReLU': nn.RReLU(),
    'SELU': nn.SELU(),
    'CELU': nn.CELU(),
    'GELU': nn.GELU(),
    'Sigmoid': nn.Sigmoid(),
    'SiLU': nn.SiLU(),
    'Mish': nn.Mish(),
    'Softplus': nn.Softplus(),
    'Softshrink': nn.Softshrink(),
    'Softsign': nn.Softsign(),
    'Tanh': nn.Tanh(),
    'Tanhshrink': nn.Tanhshrink(),
    'Threshold': nn.Threshold(),
    #'GLU': nn.GLU(),
    #'Softmin': nn.Softmin(),
    #'Softmax': nn.Softmax(),
    #'Softmax2d': nn.Softmax2d(),
    #'LogSoftmax': nn.LogSoftmax(),
    #'AdaptiveLogSoftmaxWithLoss': nn.AdaptiveLogSoftmaxWithLoss()
}

TypeError: Threshold.__init__() missing 2 required positional arguments: 'threshold' and 'value'